<a href="https://colab.research.google.com/github/Radha27112021/Python-project/blob/main/ComputerVisionSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git


Cloning into 'yolov5'...
remote: Enumerating objects: 17120, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 17120 (delta 35), reused 16 (delta 12), pack-reused 17056 (from 3)
Receiving objects: 100% (17120/17120), 15.79 MiB | 10.99 MiB/s, done.
Resolving deltas: 100% (11737/11737), done.


In [ ]:
%cd yolov5

/content/yolov5


In [ ]:
!pip install -U -r /content/yolov5/requirements.txt


In [ ]:
!pip install torch torchvision torchaudio
!pip install matplotlib opencv-python pyyaml tqdm


In [ ]:
data_yaml_content = """
train: /content/yolov5/data/images/train  # Path to training images
val: /content/yolov5/data/images/val      # Path to validation images
nc: 6  # Number of classes
names: ['person', 'car', 'truck', 'helmet', 'tire', 'door']
"""
file_path = "/content/yolov5/data/data.yaml"
#content to the file
with open(file_path, "w") as file:
    file.write(data_yaml_content)

print(f"data.yaml file created at: {file_path}")


data.yaml file created at: /content/yolov5/data/data.yaml


In [ ]:
import json
from pathlib import Path
def create_hierarchical_json(detections, class_names):
    """
    Create a hierarchical JSON structure for detections.

    Args:
        detections (list): List of detections, where each detection is [x1, y1, x2, y2, conf, cls_id].
        class_names (list): List of class names corresponding to class IDs.

    Returns:
        list: Hierarchical JSON structure.
    """
    hierarchy = []
    for det in detections:
        x1, y1, x2, y2, conf, cls_id = det
        obj = {
            "object": class_names[int(cls_id)],
            "id": f"object_{int(cls_id)}",
            "bbox": [x1, y1, x2, y2],
            "confidence": float(conf),
            "subobjects": []
        }
        # Add dummy subobjects for demonstration
        if int(cls_id) in [0, 1]:  # Example: add subobjects for 'person' or 'car'
            obj["subobjects"].append({
                "subobject": "example_subobject",
                "id": f"subobject_{int(cls_id)}_sub",
                "bbox": [x1 + 5, y1 + 5, x2 - 5, y2 - 5],
                "confidence": float(conf) * 0.9
            })
        hierarchy.append(obj)
    return hierarchy
detections = [
    [50, 50, 150, 150, 0.9, 0],  # Example detection for 'person'
    [200, 200, 300, 300, 0.8, 1]  # Example detection for 'car'
]
class_names = ['person', 'car', 'truck', 'helmet', 'tire', 'door']
hierarchical_json = create_hierarchical_json(detections, class_names)
output_path = Path("/content/detection_output.json")
with output_path.open("w") as f:
    json.dump(hierarchical_json, f, indent=4)

print(f"Hierarchical JSON saved to {output_path}")


Hierarchical JSON saved to /content/detection_output.json


In [ ]:
import cv2
import numpy as np
from pathlib import Path
# Path to save the video
video_path = "yolov5/data/video/input_video.mp4"
Path("yolov5/data/video").mkdir(parents=True, exist_ok=True)

# Video specifications
width, height = 640, 480
fps = 30
duration = 5  # seconds
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

# Create video with a moving rectangle
for i in range(fps * duration):
    frame = np.zeros((height, width, 3), dtype=np.uint8)
    cv2.rectangle(frame, (50 + i, 100), (150 + i, 200), (0, 255, 0), -1)
    out.write(frame)

out.release()
print(f"Sample video created at {video_path}")


Sample video created at yolov5/data/video/input_video.mp4


In [ ]:
import cv2
import numpy as np
import torch
import json
import time
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}")
        return

    all_detections = []  # Store detections
    frames_output = []  # Store frames for visualization

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Run inference
        results = model(frame)
        detections = results.xyxy[0].cpu().numpy()

        # Generate JSON Output for this frame
        all_detections.extend(generate_json(detections))

        # Draw bounding boxes and labels
        for det in detections:
            x1, y1, x2, y2, conf, cls = det
            label = f"Class_{int(cls)}: {conf:.2f}"
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Append processed frame (convert BGR to RGB for visualization)
        frames_output.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    cap.release()
    save_json(all_detections)  # Save detections to a JSON file
    return frames_output


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=".*autocast.*")


In [ ]:
!pip install --upgrade yolov5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.5/953.5 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 5.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114298 sha256=81

In [ ]:
# Generate JSON Output from Detections
def generate_json(detections):
    return [
        {
            "object": f"Class_{int(cls)}",
            "id": int(cls),
            "bbox": [float(x1), float(y1), float(x2), float(y2)],
            "confidence": float(conf)
        }
        for x1, y1, x2, y2, conf, cls in detections
    ]

# Save JSON Data to a File
def save_json(json_data, filename='detection_results.json'):
    with open(filename, 'w') as f:
        json.dump(json_data, f, indent=4)
    print(f"Detection results saved to {filename}")

# Display Frames in Colab Notebook
def display_frames(frames):
    plt.figure(figsize=(15, 10))
    for i, frame in enumerate(frames[:25]):
        plt.subplot(5, 5, i + 1)
        plt.imshow(frame)
        plt.axis('off')
    plt.show()

# Benchmark FPS of the Video
def benchmark(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}")
        return

    frame_count = 0
    start_time = time.time()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        model(frame)  # Run inference
        frame_count += 1

    fps = frame_count / (time.time() - start_time)
    print(f"FPS: {fps:.2f}")

# Main Execution
video_path = '/content/yolov5/data/video/input_video.mp4'  # Replace with your actual video path
frames_output = process_video(video_path)
if frames_output:
    display_frames(frames_output)
benchmark(video_path)

Detection results saved to detection_results.json
FPS: 2.82


In [ ]:
import time
start_time = time.time()
# Perform inference
end_time = time.time()
fps = 1 / (end_time - start_time)
print(f"Inference Speed: {fps} FPS")


Inference Speed: 20661.5960591133 FPS
